In [1]:
## Connection details - Edit accordingly

CLUSTER_NAME = "demo.df.io"
REMOTE_HOST = "ec2-13-40-112-81.eu-west-2.compute.amazonaws.com"
print("Done")


Done


In [4]:
%%bash
[ -f /opt/mapr/bin/mapr ] || wget https://package.mapr.hpe.com/releases/v7.0.0/mac/mapr-client-7.0.0.0.20220209033907.GA-1.x86_64.tar.gz
[ -d /opt/mapr ] || sudo tar -C /opt -xzf mapr-client-7.0.0.0.20220209033907.GA-1.x86_64.tar.gz
rm -f mapr-client-7.0.0.0.20220209033907.GA-1.x86_64.tar.gz
echo Done


Done


In [5]:
%%bash

# install x86_64 variant of brew
[ -f /usr/local/bin/brew ] || arch -x86_64 /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install.sh)"
arch -x86_64 /usr/local/bin/brew upgrade
arch -x86_64 /usr/local/bin/brew install bash gnu-getopt openssl@1.1 openjdk@11
# Create mapr user
sudo sysadminctl -addUser mapr -fullName 'DF Admin' -UID 5000 -GID 5000 -shell /usr/local/bin/bash -home /opt/mapr
# launch x86_64 shell
arch -x86_64 /usr/local/bin/bash
# Update paths/
[ -f /opt/mapr/.bashrc ] || rm /opt/mapr/.bashrc
echo '[[ -f /opt/mapr/conf/env.sh ]] && export JAVA_HOME="$(/usr/libexec/java_home) . /opt/mapr/conf/env.sh"' >> /opt/mapr/.bashrc
echo 'export PATH="/usr/local/opt/openssl@1.1/bin:$PATH"' >> /opt/mapr/.bashrc
echo 'export LDFLAGS="-L/usr/local/opt/openssl@1.1/lib"' >> /opt/mapr/.bashrc
sudo ln -sfn /usr/local/opt/openjdk@11/libexec/openjdk.jdk /Library/Java/JavaVirtualMachines/openjdk-11.jdk
sudo rm -f /usr/local/lib/libssl.1.1.dylib
sudo rm -f /usr/local/lib/libcrypto.1.1.dylib
sudo ln -s /usr/local/Cellar/openssl@1.1/1.1.1q/lib/libssl.1.1.dylib /usr/local/lib/
sudo ln -s /usr/local/Cellar/openssl@1.1/1.1.1q/lib/libcrypto.1.1.dylib /usr/local/lib/

arch -x86_64 /usr/local/bin/brew install OpenSSL@1.1
OPENSSL_INSTALLED_LOCATION=`/usr/local/bin/brew --prefix openssl@1.1`
OPENSSL_LIBRARY_PATH=${OPENSSL_INSTALLED_LOCATION}/lib
OPENSSL_PATH=${OPENSSL_INSTALLED_LOCATION}/bin
export PATH=${OPENSSL_PATH}:${PATH}
LD_LIBRARY_PATH=${OPENSSL_LIBRARY_PATH}:${LD_LIBRARY_PATH}
export JAVA_HOME=$(/usr/libexec/java_home)

echo Done


Running `brew update --auto-update`...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).

We do not provide support for this pre-release version.
You will encounter build failures with some formulae.
Please create pull requests instead of asking for help on Homebrew's GitHub,
Twitter or any other official channels. You are responsible for resolving
any issues you experience while you are running this
pre-release version.

To reinstall 5.1.16, run:
  brew reinstall bash
To reinstall 2.38, run:
  brew reinstall gnu-getopt
To reinstall 1.1.1q, run:
  brew reinstall openssl@1.1
To reinstall 11.0.16, run:
  brew reinstall openjdk@11
2022-08-03 23:39:08.450 sysadminctl[2562:2055093] User named 'mapr' already exists.
To reinstall 1.1.1q, run:
  brew reinstall openssl@1.1


Done


In [6]:
# Copy secure files

!sudo cp ssl_truststore /opt/mapr/conf/
print("Done")


Done


In [23]:
# Configure client

!echo configuring for {CLUSTER_NAME} on {REMOTE_HOST}
sedptrn = "'/^%s/d'" % CLUSTER_NAME
!sed $sedptrn /opt/mapr/conf/mapr-clusters.conf | sudo tee /opt/mapr/conf/mapr-clusters.conf

!arch -x86_64 sudo /usr/local/bin/bash /opt/mapr/server/configure.sh -c -N {CLUSTER_NAME} -C {REMOTE_HOST}:7222 -HS {REMOTE_HOST} -secure -a
!arch -x86_64 sudo /usr/local/bin/bash -c "echo mapr | /opt/mapr/bin/maprlogin password -user mapr -cluster {CLUSTER_NAME}"
print("Done")

configuring for demo.df.io on ec2-13-40-112-81.eu-west-2.compute.amazonaws.com
Node setup configuration:  hadoop-client hadoop-util
Log can be found at:  /opt/mapr/logs/configure.log
CLDB node list: ec2-13-40-112-81.eu-west-2.compute.amazonaws.com:7222
Zookeeper node list: 
External Zookeeper node list: 
Verified that OpenSSL can be successfully loaded
MapR credentials of user 'mapr' for cluster 'demo.df.io' are written to '/tmp/maprticket_0'
Done


### Client configured